# This is a modified simple tutorial using auto_ts - many thanks to Caesar for creating this notebook. Please upvote him!!
 https://www.kaggle.com/caesarlupum/bitcoin-price-prediction-auto-ts

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime

import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

## Glimpse Data

Read data

In [ ]:
path = '../input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2020-12-31.csv'
btc = pd.read_csv(path)
print(btc.shape)
btc.head()

In [ ]:
# Converting the Timestamp column from string to datetime
btc['Timestamp'] = [datetime.fromtimestamp(x) for x in btc['Timestamp']]
btc = btc.set_index('Timestamp')

#### Time resampling

In [ ]:
btc_daily = btc.resample("24H").mean() #daily resampling
print(btc_daily.shape)
btc_daily.head()

In [ ]:
btc_daily['Close'].plot(figsize=(15,8))

#### Time Series Decomposition & Statistical Tests

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
def fill_missing(df):
    ### function to impute missing values using interpolation ###
    df['Open'] = df['Open'].interpolate()
    df['Close'] = df['Close'].interpolate()
    df['Weighted_Price'] = df['Weighted_Price'].interpolate()
    df['Volume_(BTC)'] = df['Volume_(BTC)'].interpolate()
    df['Volume_(Currency)'] = df['Volume_(Currency)'].interpolate()
    df['High'] = df['High'].interpolate()
    df['Low'] = df['Low'].interpolate()
    print(df.head())
    print(df.isnull().sum())
    
fill_missing(btc_daily)

In [ ]:

plt.figure(figsize=(15,12))
series = btc_daily.Weighted_Price
result = seasonal_decompose(series, model='additive',period=1)
result.plot()

In [ ]:
daily_data_fb = btc_daily[['Close','Volume_(BTC)']]
daily_data_fb.head()

In [ ]:
test_length = 365
split_date = "2020-01-01"
#train_fb = daily_data_fb[:split_date]
#test_fb = daily_data_fb[split_date:]

print("train data percentage : %0.0f%%" %((train_fb.shape[0] / daily_data_fb.shape[0]) * 100))
print("test data percentage : %0.0f%%" %((test_fb.shape[0] / daily_data_fb.shape[0]) * 100))

![](https://github.com/AutoViML/Auto_TS/raw/master/logo.png)

[Auto_TS: Auto_TimeSeries](https://github.com/AutoViML/Auto_TS)

In [ ]:
#!pip install auto-ts

In [ ]:
from auto_ts import auto_timeseries

In [ ]:
model = auto_timeseries(score_type='rmse',
                time_interval='D', forecast_period=30,
                non_seasonal_pdq=None, seasonality=False, seasonal_period=12,
                model_type=['best'],
                verbose=2)

In [ ]:
train_fb = train_fb.reset_index()
train_fb.head()

In [ ]:
test_fb = test_fb.reset_index()
test_fb

In [ ]:
ts_column = 'Timestamp'
target = 'Close'
sep = ','

In [ ]:
model.fit(
    traindata=train_fb,
    ts_column=ts_column,
    target=target,
    cv=5,
    sep=',')

### Evaluate Results

In [ ]:
model.get_leaderboard()

In [ ]:
model.plot_cv_scores()

In [ ]:
model.get_cv_scores()

#### Forecast Results

In [ ]:
# Using Best Model
future_predictions = model.predict(
    testdata=test_fb
)  
future_predictions

In [ ]:
test_fb.head()

In [ ]:
import copy
subm = test_fb[[target]]
print(subm.shape)
subm.head()

In [ ]:
subm['predictions'] = future_predictions['yhat'].values
subm[[target,'predictions']].plot(figsize=(15,8))